In [4]:
from database.market import Market
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
import numpy as np
import statistics

In [5]:
## Loading Constants
start = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
# Loading Databases
market = Market()
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
model_range = range(len(tickers))
datasets = [
                "pdr"
            ,"tiingo"
            ,"finnhub"
           ]

In [7]:
refined_daily_classification = []
refined_quarterly_classification = []
refined_weekly_classification = []
refined_model_regression = []
market.connect()
for i in tqdm(model_range):
    try:
        refined_regression = []
        for dataset in datasets:
            ticker = tickers["Symbol"][i]
            if "." in ticker:
                ticker = ticker.replace(".","-")
            prices = market.retrieve_price_data("portfolio_{}_prices".format(dataset),ticker)
            if dataset == "pdr":
                prices = ColumnTransformer.rename_columns(prices, " ")
            else:
                prices = ColumnTransformer.rename_columns(prices, "_")
            prices = DateTransformer.convert_to_date(dataset,prices,"date")
            prices.reset_index(inplace=True)
            relev = prices[["date","adjclose"]]
            relev["ticker"] = ticker
            relev.sort_values("date",inplace=True)
            relev.rename(columns={"adjclose":dataset},inplace=True) 
            relev["date"] = [datetime.strptime(str(x).split(" ")[0],"%Y-%m-%d") for x in relev["date"]]
            ## daily transformations
            refined_regression.append(relev)
        base = refined_regression[0]
        for rr in refined_regression[1:]:
            base = base.merge(rr,on=["date","ticker"],how="left")
        adjclose = []
        for row in base.iterrows():
            values = []
            for x in datasets:
                try:
                    values.append(row[1][x])
                except:
                    continue
            adjclose.append(np.nanmean(values))
        base["adjclose"] = adjclose
        relev = base.copy()
        relev["week"] = [x.week for x in relev["date"]]
        relev["quarter"] = [x.quarter for x in relev["date"]]
        relev["year"] = [x.year for x in relev["date"]]
        refined_model_regression.append(relev.copy())
        relev_classification = relev.copy()
        relev_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_classification["adjclose"].diff()]
        refined_daily_classification.append(relev_classification)
        ## weekly transformations
        relev["week"] = [x.week for x in relev["date"]]
        relev["quarter"] = [x.quarter for x in relev["date"]]
        relev["year"] = [x.year for x in relev["date"]]
        relev_weekly_classification = relev.groupby(["year","quarter","week"]).mean().reset_index()
        relev_weekly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_weekly_classification["adjclose"].diff()]
        relev_weekly_classification["ticker"] = ticker
        refined_weekly_classification.append(relev_weekly_classification)
        ## quarterly transformations
        relev_quarterly_classification = relev.groupby(["year","quarter"]).mean().reset_index().drop("week",axis=1)
        relev_quarterly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_quarterly_classification["adjclose"].diff()]
        relev_quarterly_classification["ticker"] = ticker
        refined_quarterly_classification.append(relev_quarterly_classification)
    except Exception as e:
        print(str(e),ticker)
classification_sets = {"date":refined_daily_classification,
                       "quarter":refined_quarterly_classification,
                      "week":refined_weekly_classification}
for ds in classification_sets:
    base = pd.concat(classification_sets[ds])
    if ds == "date":
        base["year"] = [x.year for x in base["date"]]
        base["quarter"] = [x.quarter for x in base["date"]]
        base["week"] = [x.week for x in base["date"]]
        final = base.pivot_table(index=ds,values="adjclose",columns="ticker").reset_index()
    else:
        if ds == "week":
            final = base.pivot_table(index=["year","quarter","week"],values="adjclose",columns="ticker").reset_index()
        else:
            final = base.pivot_table(index=["year",ds],values="adjclose",columns="ticker").reset_index()
    name = "portfolio_dataset_{}_classification".format(ds)
    final.fillna(-99999,inplace=True)
    market.drop_table(name)
    market.store_data(name,final)
base = pd.concat(refined_model_regression)
market.drop_table("portfolio_prices")
market.store_data("portfolio_prices",base)
final = base.pivot_table(index=["year","quarter","week"],values="adjclose",columns="ticker").reset_index()
final.fillna(-99999,inplace=True)
for timeframe in ["week","quarter"]:
    if timeframe == "week":
        relev = final.groupby(["year","quarter","week"]).mean().reset_index()
    else:
        relev = final.groupby(["year","quarter"]).mean().reset_index()
    relev.reset_index(drop=True,inplace=True)
    name = "portfolio_dataset_{}_regression".format(timeframe)
    market.drop_table(name)
    market.store_data(name,relev)
market.close()

  0%|                                                                                                                                                                                     | 0/505 [00:00<?, ?it/s]<ipython-input-7-c2be8e4dbf2a>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relev["ticker"] = ticker
<ipython-input-7-c2be8e4dbf2a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relev.sort_values("date",inplace=True)
<ipython-input-7-c2be8e4dbf2a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

't' CRL


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:03<00:00,  7.91it/s]


't' YUM
't' ZBRA
't' ZBH
't' ZION
't' ZTS
